In [ ]:
# Name: Cole Linke
# Course: Computer Science Capstone
# Application: Steam Game Recommender System

# This application provides game recommendations based on a dataset of Steam games.

#------------------------------------------------------------------------------
# Import necessary libraries
#------------------------------------------------------------------------------

from IPython.display import display, HTML as display_HTML
from ipywidgets import Layout, VBox, Button, Output, Combobox, HTML, IntSlider, Tab, IntText
import matplotlib.pyplot as plt
from collections import Counter
import io
import base64
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#------------------------------------------------------------------------------
# Data Loading and Preprocessing
#------------------------------------------------------------------------------

# Load and preprocess games data from csv file
def load_games_data(games_csv_path):

    # Load CSV data
    df = pd.read_csv(games_csv_path)

    # Ensure expected columns exist
    expected_columns = [
        "AppID", "Name", "Short description", "Tags", "Genres", 
        "Positive", "Negative", "Metacritic score", "Estimated owners"
    ]
    missing = [col for col in expected_columns if col not in df.columns]
    if missing:
        raise ValueError(f"Missing expected columns: {missing}")
    
    # Rename columns for consistency
    df = df.rename(columns={
        "AppID": "app_id",
        "Name": "name",
        "Short description": "short_description",
        "Tags": "tags",
        "Genres": "genres",
        "Positive": "positive",
        "Negative": "negative",
        "Metacritic score": "metacritic_score",
        "Estimated owners": "estimated_owners"
    })

    # Ensure missing text fields don't break TF-IDF
    df["name"] = df["name"].fillna("").astype(str).str.strip()
    df["short_description"] = df["short_description"].fillna("").astype(str)

    # Convert comma-separated tags/genres into space-delimited tokens
    df["tags"] = df["tags"].fillna("").astype(str).str.replace(",", " ", regex=False)
    df["genres"] = df["genres"].fillna("").astype(str).str.replace(",", " ", regex=False)

    # Optional: collapse repeated whitespace (cleaner tokens)
    df["tags"] = df["tags"].str.replace(r"\s+", " ", regex=True).str.strip()
    df["genres"] = df["genres"].str.replace(r"\s+", " ", regex=True).str.strip()

    return df

# Prepare game features for vectorization
def prepare_game_features(df):

    # Combine relevant features into a single string
    df['combined_features'] = (
        df['name'].fillna('') + ' ' + 
        df['short_description'].fillna('') + ' ' + 
        df['tags'].fillna('') + ' ' +
        df['genres'].fillna('')
    )
    return df

# Vectorize the combined features using TF-IDF
def vectorize_game_features(df):

    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # Fit and transform the combined features
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

    return tfidf_matrix, tfidf_vectorizer

#------------------------------------------------------------------------------
# Core Recommendation Logic
#------------------------------------------------------------------------------

# Recommend similar games based on cosine similarity
def recommend_games(game_name, df, tfidf_matrix, top_n=200):

    # Check if the game exists in the dataset
    if game_name not in df['name'].values:
        return f"Game named '{game_name}' not found in the dataset."
    
    # Get the index of the target game
    game_idx = df[df['name'] == game_name].index[0]
    # Compute cosine similarity scores
    cosine_scores = cosine_similarity(
        tfidf_matrix[game_idx], 
        tfidf_matrix
    ).flatten()

    # Get the indices of the top_n most similar games (excluding itself)
    sim_scores = cosine_scores.argsort()[::-1][1:top_n+1]
    similar_games = []

    # Compile the list of similar games with their similarity percentages
    for idx in sim_scores:
        similar_games.append({
            'app_id': df.iloc[idx]['app_id'],
            'name': df.iloc[idx]['name'],
            'similarity_percent': round(cosine_scores[idx] * 100, 2),
            'metacritic_score': df.iloc[idx].get('metacritic_score'),
            'tags': df.iloc[idx]['tags'],
            'genre': df.iloc[idx]['genres']})
        if len(similar_games) == top_n:
                break

    # Return the top_n most similar games
    return similar_games

#------------------------------------------------------------------------------
# Load Data
#------------------------------------------------------------------------------

# Load and prepare data
df = load_games_data('steam_games_cleaned.csv')
df = prepare_game_features(df)
tfidf_matrix, _ = vectorize_game_features(df)

#------------------------------------------------------------------------------
# Evaluation Utilities
#------------------------------------------------------------------------------

# Convert a space-delimited string into a set of tokens
def _to_set(value):

    # Handle None or NaN values
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return set()
    
    # Ensure value is a string
    if not isinstance(value, str):
        value = str(value)

    # Strip whitespace and split into tokens
    value = value.strip()
    return set(value.split()) if value else set()

# Calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    
    union = set1 | set2
    if not union:
        return 0.0
    return len(set1 & set2) / len(union)

# Evaluate recommendation system
def evaluate_recommendations(df, tfidf_matrix, sample_size=100, k=5, sample_seed=50):

    # Initialize random number generator
    rng = np.random.default_rng(sample_seed)

    # Get indices of games with valid names
    valid_idx = df.index[df['name'].fillna('').str.strip() != ''].to_numpy()
    if len(valid_idx) == 0:
        print("No valid games with names found for evaluation.")
        return 0.0
    
    # Adjust sample size if necessary
    sample_size = min(sample_size, len(valid_idx))
    sample_idx = rng.choice(valid_idx, size=sample_size, replace=False)

    # Initialize lists to store scores
    tag_jaccard_scores = []
    genre_jaccard_scores = []
    tag_match_scores = []
    genre_match_scores = []
    avg_cosine_similarities = []

    # Counter for evaluated samples
    evaluated = 0

    # Evaluate each sampled game
    for idx in sample_idx:

        # Get target game name
        target_game = df.at[idx, 'name']

        # Get recommendations
        recs = recommend_games(target_game, df, tfidf_matrix, top_n=k)
        if isinstance(recs, str) or not recs:
            continue

        # Get target tags and genres
        target_tags = _to_set(df.at[idx, 'tags'])
        target_genres = _to_set(df.at[idx, 'genres'])

        # Initialize lists for Jaccard scores and matches
        per_tag_jaccards = []
        per_genre_jaccards = []
        cosine_values = []
        tag_matches = 0
        genre_matches = 0

        for rec in recs:

            # Get recommendation tags and genres
            rec_tags = _to_set(rec.get('tags', ""))
            rec_genres = _to_set(rec.get('genre', ""))

            # Calculate Jaccard similarity
            per_tag_jaccards.append(jaccard_similarity(target_tags, rec_tags))
            per_genre_jaccards.append(jaccard_similarity(target_genres, rec_genres))
            cosine_values.append(rec['similarity_percent'] / 100.0)

            # Check for tag match
            if target_tags & rec_tags:
                tag_matches += 1
            # Check for genre match
            if target_genres & rec_genres:
                genre_matches += 1

        # Record scores if any valid recommendations were found
        if per_tag_jaccards:
            tag_jaccard_scores.append(np.mean(per_tag_jaccards))
            genre_jaccard_scores.append(np.mean(per_genre_jaccards))
            tag_match_scores.append(tag_matches / k)
            genre_match_scores.append(genre_matches / k)
            avg_cosine_similarities.append(np.mean(cosine_values))
            evaluated += 1
    
    # Handle case with no evaluated recommendations
    if evaluated == 0:
        print("No valid recommendations could be evaluated.")
        return 0.0
    
    # Return evaluation results
    return {
        'Sample Seed': sample_seed,
        'Sample Size Requested': sample_size,
        'Sample Size Evaluated': evaluated,
        'Average Tag Jaccard Similarity': np.mean(tag_jaccard_scores),
        'Average Tag Match Rate': np.mean(tag_match_scores),
        'Average Genre Jaccard Similarity': np.mean(genre_jaccard_scores),
        'Average Genre Match Rate': np.mean(genre_match_scores),
        'Average Cosine Similarity': np.mean(avg_cosine_similarities)
    }

#------------------------------------------------------------------------------
# Visualization Utilities
#------------------------------------------------------------------------------

# Displays matplotlib figure centered on the page
def display_centered_figure(fig, margin_top=30, margin_bottom=30):
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    img_b64 = base64.b64encode(buf.read()).decode()
    display(display_HTML(f"""
        <div style="
            display: flex;
            justify-content: center;
            margin-top: {margin_top}px;
            margin-bottom: {margin_bottom}px;
        ">
            <img src="data:image/png;base64,{img_b64}" />
        </div>
    """))

#------------------------------------------------------------------------------
# UI Components
#------------------------------------------------------------------------------

# Create Title
title = HTML(
    value="""
        <div style='text-align: center; margin-bottom: 20px; font-size: 36px;'>
        <h1>Steam Game Recommender</h1>
        </div>
        """)

# Input field for game name
game_input = Combobox(
    description='',
    placeholder='Enter a game name...',
    options=[],
    layout=Layout(width='400px', align_self='center', margin='0 0 10px 0'),
    ensure_option=False)

# Input field for seed number
seed_input = IntText(
    value=50,
    description='Seed Selection:',
    layout=Layout(width='200px', align_self='center', margin='0 0 10px 0'),
    style={'description_width': 'initial'})

# Input field for minimum reviews
min_reviews_input = IntText(
    value=25,
    description='Min Reviews:',
    layout=Layout(width='200px', align_self='center', margin='0 0 10px 0'),
    style={'description_width': 'initial'})

# Recommend button
recommend_button = Button(
    description='Recommend',
    button_style='success', 
    layout=Layout(align_self='center'))

# Slider to filter metacritic score
score_slider = IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Minimum Metacritic Score:',
    continuous_update=False,
    layout=Layout(width='400px', align_self='center', margin='10px 0 20px 0'),
    style={'description_width': 'initial'})

# Button to refresh word cloud
refresh_wordcloud_button = Button(
    description='Refresh Word Cloud',
    button_style='info',
    layout=Layout(align_self='center'))

# Button to validate recommender
validation_button = Button(
    description='Validate Recommender',
    button_style='warning',
    layout=Layout(align_self='center', margin='10px 0 20px 0', width='200px'))

#------------------------------------------------------------------------------
# State Variables
#------------------------------------------------------------------------------

# Variables to store last recommendations and game name
last_recommendations = []
last_game_name = ""

#------------------------------------------------------------------------------
# Output Containers
#------------------------------------------------------------------------------

# Output widget to display recommendations
output = Output()
# Output area for word cloud
wordcloud_output = Output()
validation_output = Output()
review_distribution_output = Output()

#------------------------------------------------------------------------------
# Visualization and Plotting Functions
#------------------------------------------------------------------------------

# Plot pie chart of tags in recommended games
def plot_tag_pie_chart(recommended_games, df, top_n=10):

    # Collect all tags from recommended games
    all_tags = []

    # Collect tags from recommended games
    for game in recommended_games:

        # Get tags for the game
        game_tags = df[df['name'] == game['name']]['tags'].values

        # Append tags if available
        if len(game_tags) > 0:
            all_tags.extend(game_tags[0].split())

    # If no tags, return
    if not all_tags:
        return

    # Count tag frequencies
    tag_counts = Counter(all_tags)
    # Get the most common tags
    most_common = tag_counts.most_common(top_n)
    # Unzip labels and counts
    labels, counts = zip(*most_common)

    # Create figure
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.pie(
        counts,
        labels=labels,
        autopct='%1.1f%%',
        startangle=140
    )
    ax.set_title("Top Tags in Recommended Games")

    # Display centered figure
    display_centered_figure(fig)

# Plot histogram of estimated owners
def plot_estimated_owners_histogram(df):

    # Check if 'estimated_owners' column exists
    if 'estimated_owners' not in df.columns:
        print("Estimated owners data not available.")
        return
    
    # Define the order of estimated owners ranges
    value_order = [
        '0 - 0', '0 - 20000', '20000 - 50000', '50000 - 100000',
        '100000 - 200000', '200000 - 500000', '500000 - 1000000',
        '1000000 - 2000000', '2000000 - 5000000', '5000000 - 10000000',
        '10000000 - 20000000', '20000000 - 50000000', '50000000 - 100000000'
    ]
    
    # Count occurrences of each estimated owners range
    owners_counts = (df['estimated_owners'].value_counts().reindex(value_order, fill_value=0))

    # collapse >= 5m into one bin
    high_bin = [
        '5000000 - 10000000',
        '10000000 - 20000000',
        '20000000 - 50000000',
        '50000000 - 100000000']
    
    # merge '0 - 0' into '0 - 20000'
    owners_counts['0 - 20000'] += owners_counts.get('0 - 0', 0)
    owners_counts = owners_counts.drop('0 - 0', errors='ignore')

    # Sum high bins into one
    owners_counts['5000000+'] = owners_counts[high_bin].sum()
    owners_counts = owners_counts.drop(high_bin)

    # label bins with commas
    owners_counts.index = [
        '0 - 20,000', '20,000 - 50,000', '50,000 - 100,000',
        '100,000 - 200,000', '200,000 - 500,000', '500,000 - 1,000,000',
        '1,000,000 - 2,000,000', '2,000,000 - 5,000,000', '5,000,000+'
    ]

    # Plot histogram
    fig, ax = plt.subplots(figsize=(12, 10))
    owners_counts.plot(kind='bar', ax=ax)

    # Set titles and labels
    ax.set_title("Estimated Owners Distribution", fontsize=24, pad=20)
    ax.set_xlabel("Estimated Owners Range")
    ax.set_ylabel("Number of Games")
    plt.tick_params(axis='x', rotation=45)
    plt.tight_layout()

    # Display centered figure
    display_centered_figure(fig, margin_top=30, margin_bottom=30)

# Plot word cloud of common words in game features
def plot_wordcloud(df, max_words=150, sample_size=10000):

    # Check if 'combined_features' column exists
    if 'combined_features' not in df.columns:
        print("Combined features data not available.")
        return
    
    # Sample text data
    if len(df) > sample_size:
        text_series = df['combined_features'].dropna().sample(sample_size)
    else:
        text_series = df['combined_features'].dropna()

    # Combine all text into a single string
    text = text_series.astype(str).str.lower().str.cat(sep=' ')

    if not text:
        print("No text data available for word cloud.")
        return
    
    # Define custom stopwords
    custom_stopwords = set(STOPWORDS)
    custom_stopwords.update(['game', 'will', 'set', 'games', 'player', 'players', 'play'])
    
    # Create word cloud
    wc = WordCloud(
        width=1200,
        height=600,
        background_color='white',
        stopwords=custom_stopwords,
        max_words=max_words,
    ).generate(text)

    # Plot word cloud
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')
    ax.set_title(f"Most Common Words in Random Sample of {sample_size} Games", fontsize=24, pad=20)

    # Display centered figure
    display_centered_figure(fig, margin_top=30, margin_bottom=10)
        
# Compute positive review percentage
def compute_review_percentage(df, min_reviews = 25):

    df = df.copy()

    # Calculate total reviews
    total_reviews = df['positive'] + df['negative']

    # Filter out rows with zero negative and positive reviews
    mask_minreview = total_reviews >= min_reviews
    df = df[mask_minreview]
    total_reviews = total_reviews[mask_minreview]

    # Avoid division by zero
    mask_nonzero = total_reviews > 0
    df = df[mask_nonzero]
    total_reviews = total_reviews[mask_nonzero]

    # mask_positive = df['positive'] > 0
    # df = df[mask_positive]
    # total_reviews = total_reviews[mask_positive]

    # mask_negative = df['negative'] > 0
    # df = df[mask_negative]
    # total_reviews = total_reviews[mask_negative]

    # Calculate positive review percentage
    df['positive_review_percentage'] = (df['positive'] / total_reviews) * 100

    return df

# Plot histogram of positive review percentages
def plot_positive_review_percentage_histogram(df, bin_size=5):

    # Check if 'positive' and 'negative' columns exist
    if 'positive' not in df.columns or 'negative' not in df.columns:
        print("Positive and negative review data not available.")
        return

    # Compute positive review percentage
    df = compute_review_percentage(df, min_reviews = min_reviews_input.value)

    # Define bins
    bins = np.arange(0, 101, bin_size)
    
    # Plot histogram
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.hist(df['positive_review_percentage'], bins=bins, edgecolor='black')

    # Get current y-axis limits
    y_max = ax.get_ylim()[1] 

    # Add annotations
    ax.text(35, y_max * 0.95, 'Majority Negative Reviews', fontsize=12,
            ha='center', va='top')
    ax.text(65, y_max * 0.95, 'Majority Positive Reviews', fontsize=12,
            ha='center', va='top')

    # Set titles and labels
    ax.set_title("Positive Review Distribution", fontsize=24, pad=20)
    ax.axvline(50, color='black', linestyle='dashed', linewidth=1)
    ax.set_xlabel("Percentage of Positive Reviews")
    ax.set_ylabel("Number of Games")
    plt.xticks(np.arange(0, 101, 10))
    plt.tight_layout()

    # Display centered figure
    display_centered_figure(fig, margin_top=30, margin_bottom=30)

#------------------------------------------------------------------------------
# Recommendation Rendering Logic
#------------------------------------------------------------------------------

# Render recommendations based on last recommendations and score slider
def render_recommendations():

    # Clear previous output and display recommendations
    with output:
        output.clear_output()
        
        # Check if there are recommendations
        if not last_recommendations:
            print("No recommendations to display.")
            return
        
        # Get minimum score from slider
        min_score = score_slider.value

        # Filter recommendations based on metacritic score
        filtered = [
            g for g in last_recommendations
            if g.get('metacritic_score', 0) >= min_score
        ]

        # Display header
        display(display_HTML(f"""
                        <div style='text-align: center'>
                        <h2>'{last_game_name}'</h2>
                        <h3>Recommendations (Metacritic ≥ {min_score}):</h3>
                        </div>"""))
        
        # Display score slider
        display(VBox([score_slider], layout=Layout(align_items='center')))
        display(HTML("<div style='margin-top: 10px;'></div>"))
        
        # Check if any recommendations meet the criteria
        if not filtered:
            display(HTML(f"<div style='text-align: center;'>No recommendations meet the Metacritic score criteria.</div>"))
            return
        
        # Display top 5 recommendations
        for game in filtered[:5]:
            display(HTML(f"<div style='text-align: center;'>"
                         f"- {game['name']} (Similarity: {game['similarity_percent']}%, Metacritic: {game.get('metacritic_score', 0)})"
                         f"</div>"))
            
        # Plot tag pie chart for top 5 recommendations
        plot_tag_pie_chart(filtered[:5], df, top_n=10)

# Render word cloud
def render_wordcloud():
    # Clear previous word cloud and plot new one
    with wordcloud_output:
        wordcloud_output.clear_output()
        plot_wordcloud(df, max_words=150, sample_size=10000)

#------------------------------------------------------------------------------
# Event Handlers
#------------------------------------------------------------------------------

# Function to update game options based on user input
def update_game_options(change):

    # Get typed text
    typed_text = change['new'].strip()

    # Update options if length >= 4
    if len(typed_text) >= 4:
        # Find matching game names
        matching_games = [name for name in df['name'] if typed_text.lower() in name.lower()]
        game_input.options = matching_games[:10]  # Limit to top 10 matches
    # Clear options if input is less than 4 characters
    else:
        game_input.options = []

# Handle recommend button click
def on_recommend_button_clicked(b):

    # Get game name from input
    global last_recommendations, last_game_name

    # Clear previous output
    game_name = game_input.value.strip()
    if not game_name:
        with output:
            output.clear_output()
            print("Please enter a game name.")
        return
    
    # Get recommendations
    recommendations = recommend_games(game_name, df, tfidf_matrix, top_n=200)

    # Handle case where recommendations is an error message
    if isinstance(recommendations, str):
        with output:
            output.clear_output()
            print(recommendations)
        return
    
    # Update last recommendations and game name
    last_game_name = game_name
    last_recommendations = recommendations
    render_recommendations()

# Handle refresh word cloud button click
def on_refresh_wordcloud_button_clicked(b):
    render_wordcloud()

# Validate recommender button click
def on_validate_button_clicked(b):
    
    # Validate seed input >= 0
    if seed_input.value is None or seed_input.value < 0:
        with validation_output:
            validation_output.clear_output()
            print("Please enter a valid non-negative integer for the seed.")
        return
    
    # Clear previous validation output and run evaluation
    with validation_output:
        validation_output.clear_output()
        display(display_HTML("<div style='text-align: center;'><h3>Evaluating Recommender System...</h3></div>"))

        # Run evaluation
        results = evaluate_recommendations(df, tfidf_matrix, sample_size=100, k=5, sample_seed=seed_input.value)

        # Clear previous output
        validation_output.clear_output()

        # Display results
        if isinstance(results, dict):
            display(display_HTML("<div style='text-align: center;'><h2>Recommender System Evaluation Results</h2></div>"))
            display(display_HTML(f"""
                            <div style='text-align: center;'>
                                    <h3>Sample Seed: {results['Sample Seed']}</h3>
                                    <h3>Sample Size Requested: {results['Sample Size Requested']}</h3>
                                    <h3>Sample Size Evaluated: {results['Sample Size Evaluated']}</h3>
                                    <h3>Average Tag Jaccard Similarity: {results['Average Tag Jaccard Similarity']:.3f}</h3>
                                    <h3>Average Tag Match Rate: {results['Average Tag Match Rate']*100:.1f}%</h3>
                                    <h3>Average Genre Jaccard Similarity: {results['Average Genre Jaccard Similarity']:.3f}</h3>
                                    <h3>Average Genre Match Rate: {results['Average Genre Match Rate']*100:.1f}%</h3>
                                    <h3>Average Cosine Similarity: {results['Average Cosine Similarity']*100:.1f}%</h3>
                            </div>
                    """))
        else:
            print("Evaluation failed.")

# Handle score slider change
def on_score_slider_changed(change):
    # Re-render recommendations when score slider changes
    if last_recommendations:
        render_recommendations()

# Handle min reviews input change
def on_min_reviews_input_changed(change):

    with review_distribution_output:
        review_distribution_output.clear_output()
        plot_positive_review_percentage_histogram(df, bin_size=5)

#------------------------------------------------------------------------------
# Widget Observers and Event Bindings
#------------------------------------------------------------------------------

# Observe changes in the input field
game_input.observe(update_game_options, names='value')
# Observe changes in score slider
score_slider.observe(on_score_slider_changed, names='value')
# Observe changes in min reviews input
min_reviews_input.observe(on_min_reviews_input_changed, names='value')
# Bind button click event
recommend_button.on_click(on_recommend_button_clicked)
# Bind refresh word cloud button click event
refresh_wordcloud_button.on_click(on_refresh_wordcloud_button_clicked)
# Bind validate recommender button click event
validation_button.on_click(on_validate_button_clicked)

#------------------------------------------------------------------------------
# Tab Layout and Assembly
#------------------------------------------------------------------------------

# Display UI with Tabs
# Tab 1: Visuals
visuals_output = Output()

# Plot estimated owners histogram
with visuals_output:
    plot_estimated_owners_histogram(df)

with review_distribution_output:
    plot_positive_review_percentage_histogram(df, bin_size=5)

# Create visuals tab
visuals_title = HTML(
    value="""
        <div style='text-align: center; margin-bottom: 30px; font-size: 36px;'>
        <h1>Steam Game Visualizations</h1>
        </div>
        """)

visuals_tab = VBox(
    [visuals_title, visuals_output, review_distribution_output, min_reviews_input, wordcloud_output, refresh_wordcloud_button],
    layout=Layout(align_items='stretch'))

# Tab 2: Recommender
recommender_tab = VBox(
    [title, game_input, recommend_button, output],
    layout=Layout(align_items='center'))

# Tab 3: Validation
validation_tab = VBox(
    [seed_input, validation_button, validation_output],
    layout=Layout(align_items='center'))

# Create Tabs
tabs = Tab(children=[recommender_tab, visuals_tab, validation_tab])
tabs.set_title(0, 'Recommender')
tabs.set_title(1, 'Visuals')
tabs.set_title(2, 'Validation')

#------------------------------------------------------------------------------
# Initial Rendering
#------------------------------------------------------------------------------

render_wordcloud()
display(tabs)

#------------------------------------------------------------------------------
# Resources
#------------------------------------------------------------------------------
# 1. Steam Game Dataset: https://www.kaggle.com/datasets/fronkongames/steam-games-dataset
# 2. TF-IDF Vectorization: https://scikit-learn.org/stable/modules/feature_extraction.html
# 3. Cosine Similarity: https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
# 4. Jaccard Similarity: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html & https://en.wikipedia.org/wiki/Jaccard_index
# 5. Word Cloud Generation: https://amueller.github.io/word_cloud/
# 6. IPyWidgets Documentation: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
# 7. Matplotlib Documentation: https://matplotlib.org/stable/users/index
# 8. Pandas Documentation: https://pandas.pydata.org/docs/user_guide/index.html#user-guide
# 9. NumPy Documentation: https://numpy.org/doc/2.4/user/index.html#user
# 10. Content-Based Recommender System: https://www.geeksforgeeks.org/machine-learning/ml-content-based-recommender-system/
# 11. Content-Based Recommender System: https://www.kaggle.com/code/ramzanzdemir/recommendation-systems-content-based-tf-idf
# 12. Pie Chart Centering: Pie chart centering: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html

        


                                 name    genres                                                                                                                                              tags
Supipara - Chapter 1 Spring Has Come! Adventure                                                                                                                 Adventure Visual Novel Anime Cute
    Mystery Solitaire The Black Raven    Casual Casual Card Game Solitaire Puzzle Hidden Object 2D Colorful Stylized Logic Mystery Atmospheric Family Friendly PvE Tutorial Singleplayer Tabletop
